In [ ]:
import numpy as np
from suite2p import run_s2p
from suite2p.io import BinaryFile
from suite2p.detection import bin_movie
from pathlib import Path
import matplotlib.pyplot as plt


run to get intermediate data during detection

In [ ]:

root = Path('/media/carsen/disk2/suite2p_paper/GT1/')
settings = np.load(root / 'suite2p/plane1/settings.npy', allow_pickle=True).item()
db = np.load(root / 'suite2p/plane1/db.npy', allow_pickle=True).item()
reg_outputs = np.load(root / 'suite2p/plane1/reg_outputs.npy', allow_pickle=True).item()
reg_file = root / 'suite2p/plane1/data.bin'

tau = settings['tau']
fs = settings['fs']
Ly, Lx = db['Ly'], db['Lx']
with BinaryFile(Ly=Ly, Lx=Lx, filename=reg_file) as f_reg:
    n_frames, Ly, Lx = f_reg.shape
    yrange = reg_outputs['yrange']
    xrange = reg_outputs['xrange']
    badframes = reg_outputs['badframes']
    nbins = settings['detection']['nbins']
    bin_size = int(max(1, n_frames // nbins, np.round(tau * fs)))
    mov = bin_movie(f_reg, bin_size, yrange=yrange, xrange=xrange,
                        badframes=badframes, nbins=nbins)

mov_ex = mov[:100].copy()
print(mov.shape)

tau = settings['tau']
fs = settings['fs']
Ly, Lx = db['Ly'], db['Lx']
with BinaryFile(Ly=Ly, Lx=Lx, filename=reg_file) as f_reg:
    n_frames, Ly, Lx = f_reg.shape
    yrange = reg_outputs['yrange']
    xrange = reg_outputs['xrange']
    badframes = reg_outputs['badframes']
    nbins = settings['detection']['nbins']
    bin_size = int(max(1, n_frames // nbins, np.round(tau * fs)))
    mov = bin_movie(f_reg, bin_size, yrange=yrange, xrange=xrange,
                        badframes=badframes, nbins=nbins)

mov_ex = mov[:100].copy()

from suite2p.detection import utils
from suite2p.detection.sparsedetect import neuropil_subtraction

mov = utils.temporal_high_pass_filter(mov=mov, width=settings['detection']["highpass_time"])
sdmov = utils.standard_deviation_over_time(mov, batch_size=1000)
mov = neuropil_subtraction(
        mov=mov / sdmov,
        filter_size=settings['detection']['sparsery_settings']['highpass_neuropil'])  # subtract low-pass filtered movie

mov_filt_ex = mov.copy()

from suite2p.detection.sparsedetect import square_convolution_2d, downsample
_, Lyc, Lxc = mov.shape
LL = np.meshgrid(np.arange(Lxc), np.arange(Lyc))
gxy = [np.array(LL).astype("float32")]
dmov = mov
movu = []

# downsample movie at various spatial scales
Lyp, Lxp = np.zeros(5, "int32"), np.zeros(5, "int32")  # downsampled sizes
for j in range(5):
    movu0 = square_convolution_2d(dmov, 3)
    dmov = 2 * downsample(dmov)
    gxy0 = downsample(gxy[j], False)
    gxy.append(gxy0)
    _, Lyp[j], Lxp[j] = movu0.shape
    movu.append(movu0)

from suite2p.detection.sparsedetect import find_best_scale, threshold_reduce
from scipy.interpolate import RectBivariateSpline

spatial_scale = settings['detection']['sparsery_settings']['spatial_scale']
threshold_scaling = settings['detection']['threshold_scaling']

# spline over scales
I = np.zeros((len(gxy), gxy[0].shape[1], gxy[0].shape[2]))
for movu0, gxy0, I0 in zip(movu, gxy, I):
    gmodel = RectBivariateSpline(gxy0[1, :, 0], gxy0[0, 0, :], movu0.max(axis=0),
                                    kx=min(3, gxy0.shape[1] - 1),
                                    ky=min(3, gxy0.shape[2] - 1))
    I0[:] = gmodel(gxy[0][1, :, 0], gxy[0][0, 0, :])
v_corr = I.max(axis=0)

scale, estimate_mode = find_best_scale(I=I, spatial_scale=spatial_scale)
# TODO: scales from cellpose (?)
#    scales = 3 * 2 ** np.arange(5.0)
#    scale = np.argmin(np.abs(scales - diam))
#    estimate_mode = EstimateMode.Estimated

spatscale_pix = 3 * 2**scale
mask_window = int(((spatscale_pix * 1.5) // 2) * 2)
Th2 = threshold_scaling * 5 * max(
    1, scale)  # threshold for accepted peaks (scale it by spatial scale)
vmultiplier = max(1, mov.shape[0] / 1200)
print("NOTE: %s spatial scale ~%d pixels, time epochs %2.2f, threshold %2.2f " %
        (estimate_mode.value, spatscale_pix, vmultiplier, vmultiplier * Th2))

# get standard deviation for pixels for all values > Th2
v_map = [threshold_reduce(movu0, Th2) for movu0 in movu]

from suite2p.detection.sparsedetect import add_square, iter_extend, two_comps, multiscale_mask, extendROI
from copy import deepcopy
import time

max_ROIs = 10000
active_percentile = 0

movu = [movu0.reshape(movu0.shape[0], -1) for movu0 in movu]

mov = np.reshape(mov, (-1, Lyc * Lxc))
lxs = 3 * 2**np.arange(5)
nscales = len(lxs)

v_max = np.zeros(max_ROIs)
ihop = np.zeros(max_ROIs)
v_split = np.zeros(max_ROIs)
V1 = deepcopy(v_map)
stats = []
patches = []
seeds = []
extract_patches = False

t0 = time.time()
ypix_all = []
xpix_all = []
lam_all = []
tproj_all = []
active_frames_all = []
f_init = []
for tj in range(max_ROIs):
    # find peaks in stddev"s
    v0max = np.array([V1[j].max() for j in range(5)])
    imap = np.argmax(v0max)
    imax = np.argmax(V1[imap])
    yi, xi = np.unravel_index(imax, (Lyp[imap], Lxp[imap]))
    # position of peak
    yi, xi = gxy[imap][1, yi, xi], gxy[imap][0, yi, xi]
    med = [int(yi), int(xi)]

    # check if peak is larger than threshold * max(1,nbinned/1200)
    v_max[tj] = v0max.max()
    if v_max[tj] < vmultiplier * Th2:
        break
    ls = lxs[imap]

    ihop[tj] = imap

    # make square of initial pixels based on spatial scale of peak
    yi, xi = int(yi), int(xi)
    ypix0, xpix0, lam0 = add_square(yi, xi, ls, Lyc, Lxc)

    # project movie into square to get time series
    tproj = (mov[:, ypix0 * Lxc + xpix0] * lam0[0]).sum(axis=-1)
    f_init.append(tproj.copy())
    if active_percentile > 0:
        threshold = min(Th2, np.percentile(tproj, active_percentile))
    else:
        threshold = Th2
    active_frames = np.nonzero(tproj > threshold)[0]  # frames with activity > Th2

    # get square around seed
    if extract_patches:
        mask = mov[active_frames].mean(axis=0).reshape(Lyc, Lxc)
        patches.append(utils.square_mask(mask, mask_window, yi, xi))
        seeds.append([yi, xi])

    # extend mask based on activity similarity
    ypixs, xpixs = [ypix0.copy()], [xpix0.copy()]
    lams = [np.ones(len(ypix0), 'float32')]
        
    for j in range(3):
        #ypix0, xpix0, lam0 = iter_extend(ypix0, xpix0, mov, Lyc, Lxc, active_frames)
        npix = 0
        iter = 0
        ypix, xpix = ypix0.copy(), xpix0.copy()
        while npix < 10000:
            npix = ypix.size
            # extend ROI by 1 pixel on each side
            ypix, xpix = extendROI(ypix, xpix, Lyc, Lxc, 1)
            # activity in proposed ROI on ACTIVE frames
            usub = mov[np.ix_(active_frames, ypix * Lxc + xpix)]
            lam = np.mean(usub, axis=0)
            ix = lam > max(0, lam.max() / 5.0)
            if ix.sum() == 0:
                break
            ypix, xpix, lam = ypix[ix], xpix[ix], lam[ix]
            if iter == 0:
                sgn = 1.
            if np.sign(sgn * (ix.sum() - npix)) <= 0:
                break
            else:
                npix = ypix.size
            iter += 1
            ypixs.append(ypix.copy())
            xpixs.append(xpix.copy())
            lams.append(lam.copy())
        lam = lam / np.sum(lam**2)**.5
        ypix0, xpix0, lam0 = ypix.copy(), xpix.copy(), lam.copy()
        tproj = mov[:, ypix0 * Lxc + xpix0] @ lam0
        active_frames = np.nonzero(tproj > threshold)[0]
        if len(active_frames) < 1:
            #if tj < max_ROIs/2: # TODO: nmasks is undefined
            #    continue
            #else:
            break
    
    if len(active_frames) < 1:
        #if tj < max_ROIs/2:
        #    continue
        #else:
        break

    ypix_all.append(ypixs)
    xpix_all.append(xpixs)
    lam_all.append(lams)
    
    # check if ROI should be split
    v_split[tj], ipack = two_comps(mov[:, ypix0 * Lxc + xpix0], lam0, threshold)
    if v_split[tj] > 1.25:
        lam0, xp, active_frames = ipack
        tproj[active_frames] = xp
        ix = lam0 > lam0.max() / 5
        xpix0 = xpix0[ix]
        ypix0 = ypix0[ix]
        lam0 = lam0[ix]
        ymed = np.median(ypix0)
        xmed = np.median(xpix0)
        imin = np.argmin((xpix0 - xmed)**2 + (ypix0 - ymed)**2)
        med = [ypix0[imin], xpix0[imin]]

    # update residual on raw movie
    mov[np.ix_(active_frames,
                ypix0 * Lxc + xpix0)] -= tproj[active_frames][:, np.newaxis] * lam0
    # update filtered movie
    ys, xs, lms = multiscale_mask(ypix0, xpix0, lam0, Lyp, Lxp)
    for j in range(nscales):
        movu[j][np.ix_(active_frames, xs[j] + Lxp[j] * ys[j])] -= np.outer(
            tproj[active_frames], lms[j])
        Mx = movu[j][:, xs[j] + Lxp[j] * ys[j]]
        V1[j][ys[j], xs[j]] = (Mx**2 * np.float32(Mx > threshold)).sum(axis=0)**.5

    tproj_all.append(tproj)
    active_frames_all.append(active_frames)

    stats.append({
        "ypix": ypix0.astype(int),
        "xpix": xpix0.astype(int),
        "lam": lam0 * sdmov[ypix0, xpix0],
        "med": med,
        "footprint": ihop[tj]
    })

    if tj % 500 == 0:
        t1 = time.time() - t0
        print(f"ROIs: {tj},\t last score: {v_max[tj]:0.4f}, \t time: {t1:0.2f}sec")

import cv2
v_map_rsz = [cv2.resize(vm, (Lxc, Lyc), interpolation=cv2.INTER_LINEAR) 
             for vm in v_map]
mov = mov.reshape(-1, Lyc, Lxc)
ihop = ihop[:len(stats)]

masks_overlap = np.zeros((Lyc, Lxc), 'int')
masks = np.zeros((Lyc, Lxc), 'uint16')
masks_all = [np.zeros((Lyc, Lxc), 'uint16') for i in range(len(v_map))]
iperm = np.random.permutation(len(stats))
for n in range(len(stats)-1, -1, -1):
    ypix, xpix = stats[n]['ypix'], stats[n]['xpix']
    #overlap = stat[n]['overlap']
    #ypix, xpix = ypix[~overlap], xpix[~overlap]
    masks_all[int(ihop[n])][ypix, xpix] = iperm[n]+1
    masks_overlap[ypix, xpix] += 1
    masks[ypix, xpix] = n+1 #iperm[n]+1#n+1

ioverlap = np.unique(masks[10:50, 320:360])[1:] - 1
                     #[110:135, 135:165])[1:] - 1 
#215:235, -89:-80])[1:] - 1
print(ioverlap)
print([(masks_overlap[stats[i]['ypix'], stats[i]['xpix']] > 1).mean() for i in ioverlap])

overlap = np.zeros((Lyc, Lxc), 'uint16')
mask_id = np.zeros((len(ioverlap), Lyc, Lxc), 'bool')
mask_pic = np.ones((Lyc, Lxc, 3), 'float32')
colors = plt.get_cmap('tab20')
colors = plt.get_cmap('gist_ncar')(np.linspace(0, 0.9, len(ioverlap)))[::-1]
traces = np.zeros((len(ioverlap), len(mov)))
for i, ic in enumerate(ioverlap):
    overlap[stats[ic]['ypix'], stats[ic]['xpix']] += 1 
    mask_id[i, stats[ic]['ypix'], stats[ic]['xpix']] = True
    mask_pic[mask_id[i]] = colors[i][:3]
    f = tproj_all[ic].copy()
    f -= f.min()
    f /= f.max()
    traces[i, active_frames_all[ic]] = f[active_frames_all[ic]]

for iy, ix in zip(np.nonzero(overlap > 1)[0], np.nonzero(overlap > 1)[1]):
    ii = np.nonzero(mask_id[:, iy, ix])[0]
    col = np.array([colors[i][:3] for i in ii]).mean(axis=0)
    col += 0.1
    col = np.minimum(1., col)
    mask_pic[iy, ix] = col#ors(i)[:3]
    
#pic[overlap > 1] = 0.8 * np.ones(3)    
intersection = ((traces > 0).astype('int') @ (traces > 0).astype('int').T).astype('float32')
union = (traces > 0).sum(axis=1) + (traces > 0).sum(axis=1)[:,np.newaxis] - intersection #/ ((traces > 0) + (traces > 0).T)
iou = intersection / union
iou[np.triu_indices(iou.shape[0], k=0)[0], np.triu_indices(iou.shape[0], k=0)[1]] = np.nan


# olims = [[199, 236], [528, 561]]
olims = [[np.nonzero(mask_id.sum(axis=(0,2)))[0][0], Lyc - np.nonzero(mask_id.sum(axis=(0,2))[::-1])[0][0]-1],
         [np.nonzero(mask_id.sum(axis=(0,1)))[0][0], Lxc - np.nonzero(mask_id.sum(axis=(0,1))[::-1])[0][0]-1]]
olims[0][1] -= 20
olims[1][0] += 0
olims[1][1] -= 20
print(olims)


In [ ]:
np.save('results/ex_detect.npy', {'mov': mov_ex, 'mov_filt': mov_filt_ex, 'v_map': v_map_rsz, 
                                  'ypix_all': ypix_all, 'xpix_all': xpix_all, 
                                  'lam_all': lam_all, 'f_init': f_init, 'threshold': threshold, 
                                  'masks_all': masks_all, 'mask_pic': mask_pic, 'mask_id': mask_id,
                                  'iou': iou, 'traces': traces, 'colors': colors})

In [ ]:
dat = np.load('results/ex_detect.npy', allow_pickle=True).item()

### figure

In [ ]:
import figures
import importlib 
importlib.reload(figures)

ylim = [0, 450]
xlim = [300, 622]
fig = figures.detection_fig(ylim, xlim, **dat)
fig.savefig('figures/fig4.pdf', dpi=150)

# benchmark

first run suite2p for registration and `max_proj`, and extract cells using Cellpose from `max_proj` image:

In [ ]:
from pathlib import Path
import numpy as np
from suite2p import default_settings, default_db, parameters
import torch 
import detect_benchmarks 
from suite2p import run_s2p
device = torch.device("cuda")

# set to path to TIFFS
root = Path("/media/carsen/disk1/suite2p_paper/VG23/2025_08_06/1/")
n_planes = 4
settings = default_settings()
db = default_db()
import json
with open(root / "ops.json", "r") as f:
    settings_in = json.load(f)

db, settings, settings_in = parameters.convert_settings_orig(settings_in, 
                                                          db=db, 
                                                          settings=settings)

db['data_path'] = [str(root)]
db["save_path0"] = str(root) 
db["nplanes"] = n_planes
settings['tau'] = 0.25
settings['run']['multiplane_parallel'] = False
settings['io']['delete_bin'] = False

print(db['fast_disk'])

run_s2p(db=db, settings=settings);

# remove baseline fluorescence
for ipl in range(n_planes):
    detect_benchmarks.baseline_movie(root, ipl=ipl)

# downsample planes spatially
for ipl in range(n_planes):
    detect_benchmarks.downsample_movie(root, ipl=ipl, ds=2, do_filt=True)

### cellpose segmentation on full movies
# get traces from downsampled orig movie and baselined movie
for ipl in range(4):
    db = np.load(root / "suite2p" / f"plane{ipl}" / "db.npy", allow_pickle=True).item()
    Ly, Lx = db["Ly"], db["Lx"]
    reg_outputs = np.load(root / "suite2p" / f"plane{ipl}" / "reg_outputs.npy", allow_pickle=True).item()
    yrange, xrange = reg_outputs["yrange"], reg_outputs["xrange"]
    meanImg = reg_outputs["meanImg"]
    detect_outputs = np.load(root / "suite2p" / f"plane{ipl}" / "detect_outputs.npy", allow_pickle=True).item()
    max_proj0 = detect_outputs["max_proj"]
    max_proj = np.zeros((Ly, Lx), dtype="float32")
    max_proj[yrange[0] : yrange[1], xrange[0] : xrange[1]] = max_proj0
    meanImg[:yrange[0], :] = 0
    meanImg[yrange[1]:, :] = 0
    meanImg[:, :xrange[0]] = 0
    meanImg[:, xrange[1]:] = 0

    img = max_proj.copy()

    masks, stat_gt, F_gt, Fneu_gt = detect_benchmarks.cellpose_extract(img, Ly, Lx, 
                                                     [root / "suite2p_ds" / f"plane{ipl}" / "data.bin", 
                                                     root / "suite2p_ds" / f"plane{ipl}" / "data_filt.bin"], ds=2)
    print(masks.max())

    (root / "benchmarks").mkdir(exist_ok=True)
    np.save(root / "benchmarks" / f"stat_gt_plane{ipl}.npy", stat_gt)
    np.save(root / "benchmarks" / f"F_gt_plane{ipl}.npy", F_gt[0])
    np.save(root / "benchmarks" / f"F_gt_filt_plane{ipl}.npy", F_gt[1])
    np.save(root / "benchmarks" / f"Fneu_gt_plane{ipl}.npy", Fneu_gt[0])
    np.save(root / "benchmarks" / f"Fneu_gt_filt_plane{ipl}.npy", Fneu_gt[1])
    np.save(root / "benchmarks" / f"masks_gt_plane{ipl}.npy", masks)
    np.save(root / "benchmarks" / f"img_gt_plane{ipl}.npy", img)

### generate hybrid ground-truth + run suite2p

For the `threshold_scaling` sweep, run the following in the env, changing path to binaries for each mouse to folder containing `suite2p_ds`:
```
python detect_benchmarks.py --root /path/to/bin/ --param_sweep --iplane 0
python detect_benchmarks.py --root /path/to/bin/ --param_sweep --iplane 1
python detect_benchmarks.py --root /path/to/bin/ --param_sweep --iplane 2
python detect_benchmarks.py --root /path/to/bin/ --param_sweep --iplane 3
```

Then for the sweep across different simulation parameters, run:
```
python detect_benchmarks.py --root /path/to/bin/ --sweep --iplane 0
python detect_benchmarks.py --root /path/to/bin/ --sweep --iplane 1
python detect_benchmarks.py --root /path/to/bin/ --sweep --iplane 2
python detect_benchmarks.py --root /path/to/bin/ --sweep --iplane 3
```


### run caiman

Create an env with caiman:

```
conda create --name caiman caiman -c conda-forge -y
conda activate caiman
pip install natsort
```

For the $K$ num components sweep, run the following in the env, changing path to binaries for each mouse:
```
python detect_caiman.py --root /path/to/bin/ --param_sweep --iplane 0
python detect_caiman.py --root /path/to/bin/ --param_sweep --iplane 1
python detect_caiman.py --root /path/to/bin/ --param_sweep --iplane 2
python detect_caiman.py --root /path/to/bin/ --param_sweep --iplane 3
```

For the grid sweep, run the following:
```
python detect_caiman.py --root /path/to/bin/ --param_sweep_grid --iplane 0
python detect_caiman.py --root /path/to/bin/ --param_sweep_grid --iplane 1
python detect_caiman.py --root /path/to/bin/ --param_sweep_grid --iplane 2
python detect_caiman.py --root /path/to/bin/ --param_sweep_grid --iplane 3
```


Then for the sweep across different simulations, run:
```
python detect_caiman.py --root /path/to/bin/ --sweep --iplane 0
python detect_caiman.py --root /path/to/bin/ --sweep --iplane 1
python detect_caiman.py --root /path/to/bin/ --sweep --iplane 2
python detect_caiman.py --root /path/to/bin/ --sweep --iplane 3
```

### compute optimal threshold / comps for suite2p / caiman

and create supp fig

In [ ]:
ths = np.arange(0.6, 1.5, 0.1)

tps = np.zeros((3, 4, len(ths)))
fps = np.zeros((3, 4, len(ths)))
fns = np.zeros((3, 4, len(ths)))
n_ell, neu_coeff, poisson_coeff = 2000, 0.4, 20

for im, mname in enumerate(['VG21', 'VG23', 'VG24']):
    root = Path(f'/home/carsen/dm11_string/suite2p_paper/hybrid_gt/{mname}/')
    for iplane in range(4):
        for i, th in enumerate(ths):
            rstr = f's2p_{th:.1f}'
            filename = root / 'sims' / f'results_{rstr}_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}.npy'
            # print(filename)
            if filename.exists():
                #print(f'Loading {filename}')
                tp, fp, fn, f1 = np.load(filename)
                tps[im, iplane, i] = tp
                fps[im, iplane, i] = fp
                fns[im, iplane, i] = fn

f1s = tps / (tps + 0.5*(fns+fps))
print(f1s.mean(axis=(0, 1)))

metrics_s2p = [f1s, tps, fns, fps]
n_ell, neu_coeff, poisson_coeff = 2000, 0.4, 20
K = np.array([7,8,9,10,11,12,13])
tps = np.zeros((3, 4, len(K)))
fps = np.zeros((3, 4, len(K)))
fns = np.zeros((3, 4, len(K)))
for im, mname in enumerate(['VG21', 'VG23', 'VG24']):
    root = Path(f'/home/carsen/dm11_string/suite2p_paper/hybrid_gt/{mname}/')
    for iplane in range(4):
        for i, K0 in enumerate(K):
            rstr = f'caiman_K_{K0}_p_1_gnb_2_gSig0_5_rf_15'
            #print(rstr)
            filename = root / 'sims' / f'results_{rstr}_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}.npy'
            # print(filename)
            if filename.exists():
                #print(f'Loading {filename}')
                tp, fp, fn, f1 = np.load(filename)
                tps[im, iplane, i] = tp
                fps[im, iplane, i] = fp
                fns[im, iplane, i] = fn
            else:
                print(filename)

f1s = tps / (tps + 0.5*(fns+fps))

metrics_caiman = [f1s, tps, fns, fps]

print(f1s.mean(axis=(0,1)))

from fig_utils import *
fig = plt.figure(figsize=(14,3), dpi=150)
grid = plt.GridSpec(1, 6, wspace=0.5, hspace=0.5, figure=fig, 
                            bottom=0.2, top=0.88, left=0.05, right=0.99)

ylabels = ['F1 score', 'false negatives', 'false positives']
il = 0
transl = mtransforms.ScaledTranslation(-40/72, 5/72, fig.dpi_scale_trans)
xlabels = ['threshold scaling', '# of components (K)']
for i, (params, metrics) in enumerate(zip([ths, K], [metrics_s2p, metrics_caiman])):
    for j in range(3):
        ax = plt.subplot(grid[0, 3*i + j])
        metric = metrics[j + 1*(j>0)].copy()
        mmean = metric.mean(axis=(0,1))
        merr = metric.std(axis=(0,1)) / (np.prod(metric.shape[:2])-1)**0.5
        ax.errorbar(params, mmean, merr, color=alg_cols[i])
        if j==0:
            ibest = mmean.argmax()
            ax.set_ylim([0, 0.9])
            il = plot_label(ltr, il, ax, transl) 
        else:
            ax.set_ylim([0, 1000])
        ax.scatter(params[ibest], mmean[ibest], marker='*', color='k', 
                       s=100, alpha=1, zorder=0)
        ax.set_ylabel(ylabels[j])
        ax.set_xlabel(xlabels[i])
        if i==1:
            ax.set_xticks([8, 10, 12])
        if j==0:
            ax.set_title(alg_names[i][:1].upper()+alg_names[i][1:], color=alg_cols[i],
                         loc='left')
fig.savefig('figures/suppfig_thK.pdf')

### aggregate TPs, FPs, FNs from caiman parameter grid sweep

and create supp fig

In [ ]:
tps_all = np.zeros((3,4,2,3,3,3,3))
fps_all = np.zeros((3,4,2,3,3,3,3))
fns_all = np.zeros((3,4,2,3,3,3,3))

n_ell, neu_coeff, poisson_coeff = 2000, 0.4, 20
for im, mname in enumerate(['VG21', 'VG23', 'VG24']):
    root = Path(f'/home/carsen/dm11_string/suite2p_paper/hybrid_gt/{mname}/')

    for iplane in range(4):
        tps, fps, fns = [], [], []
        for p in [1, 2]:
            for gnb in [1, 2, 3]:
                for K in [6, 9, 12]:
                    for gSig0 in [3, 5, 7]:
                        for rf in [12, 15, 18]:
                            rstr = f'caiman_K_{K}_p_{p}_gnb_{gnb}_gSig0_{gSig0}_rf_{rf}'
                            filename = root / 'sims' / f'results_{rstr}_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}.npy'
                            if filename.exists():
                                #print(f'Loading {filename}')
                                tp, fp, fn, f1 = np.load(filename)
                                tps.append(tp)
                                fps.append(fp)
                                fns.append(fn)
                            else:
                                print(filename)
        tps = np.array(tps)
        fps = np.array(fps)
        fns = np.array(fns)

        tps = tps.reshape(2,3,3,3,3)
        fps = fps.reshape(2,3,3,3,3)
        fns = fns.reshape(2,3,3,3,3)
        tps_all[im, iplane] = tps
        fps_all[im, iplane] = fps
        fns_all[im, iplane] = fns

from fig_utils import *

param_names = ['autoreg order (p)', '# of background comps. (gnb)', 
               '# of components (K)', 'gaussian smoothing (gSig)', 'patch half-size (rf)']
params = [[1, 2], [1, 2, 3], [6, 9, 12], [3, 5, 7], [12, 15, 18]]

fig = plt.figure(figsize=(10,8), dpi=150)
grid = plt.GridSpec(3, 5, wspace=0.4, hspace=0.1, figure=fig, 
                            bottom=0.05, top=0.92, left=0.07, right=0.99)

f1s_all = tps_all.copy() / (tps_all + 0.5 * (fps_all + fns_all))
metrics_orig = [[metrics_s2p[k][:,:,metrics_s2p[0].mean(axis=(0,1)).argmax()].flatten(), 
                 metrics_caiman[k][:,:,metrics_caiman[0].mean(axis=(0,1)).argmax()].flatten()] 
                 for k in [0, 2, 3]]
il = 0
transl = mtransforms.ScaledTranslation(-40/72, 0/72, fig.dpi_scale_trans)
for i in range(5):
    ni = np.arange(2, 7)
    ni = np.delete(ni, i)
    freshape = f1s_all.copy().transpose(0, 1, i+2, *ni).reshape(np.prod(f1s_all.shape[:2]), f1s_all.shape[i+2], -1)
    print(freshape.shape)
    imax = freshape.mean(axis=0).argmax(axis=-1)
    nl = freshape.shape[1]
    for k in range(3):
        ax = plt.subplot(grid[k, i])
        if k==0:
            pos = ax.get_position().bounds 
            ax.set_position([pos[0], pos[1]+0.04, *pos[2:]])
        if k==1:
            freshape = fns_all.copy().transpose(0, 1, i+2, *ni).reshape(np.prod(f1s_all.shape[:2]), f1s_all.shape[i+2], -1)
        elif k==2:
            freshape = fps_all.copy().transpose(0, 1, i+2, *ni).reshape(np.prod(f1s_all.shape[:2]), f1s_all.shape[i+2], -1)
        fmax = freshape[:, np.arange(nl), imax]
        fmean = fmax.mean(axis=0)
        nerr = (fmax.shape[0] - 1)**0.5
        ferr = fmax.std(axis=0) / nerr
        ax.errorbar(params[i], fmean, ferr, color='k')
        for j, metric in enumerate(metrics_orig[k]):
            ax.plot(params[i], metric.mean() * np.ones(nl), color=alg_cols[j], ls='--', zorder=30, lw=2)
        if i==2:
            ax.set_xticks([6, 8, 10, 12])
        elif i==0:
            ax.set_xticks([1, 2])
            ax.set_ylabel(ylabels[k])
        if k==0:
            ax.set_xlabel(param_names[i])
            ax.set_ylim([0.3, 0.8])
            il = plot_label(ltr, il, ax, transl) 
        else:
            ax.set_ylim([0, 720])
        if i==0 and k==0:
            ax.text(0.05, 0.83, alg_names[0][:1].upper()+alg_names[0][1:], color=alg_cols[0],
                    transform=ax.transAxes)
            ax.text(0.05, 0.04, alg_names[1][:1].upper()+alg_names[1][1:] + 
                    '\n(p=1, gnb=2, K=9, \n gSig=5, rf=15)', 
                    color=alg_cols[1], transform=ax.transAxes)

fig.savefig('figures/suppfig_sweep.pdf')

### aggregate TPs, FPs, FNs for all simulations

In [ ]:
from pathlib import Path
n_ells = np.arange(0, 4001, 500)
neu_coeffs = np.arange(0, 0.81, 0.1)
poisson_coeffs = [0, 5, 10, 20, 50, 100, 200]

tps = [np.zeros((3, 4, 2, len(var))) for var in [n_ells, neu_coeffs, poisson_coeffs]]
fps = [np.zeros((3, 4, 2, len(var))) for var in [n_ells, neu_coeffs, poisson_coeffs]]
fns = [np.zeros((3, 4, 2, len(var))) for var in [n_ells, neu_coeffs, poisson_coeffs]]

for im, mname in enumerate(['VG21', 'VG23', 'VG24']):
    root = Path(f'/home/carsen/dm11_string/suite2p_paper/hybrid_gt/{mname}/')
    for j, var in enumerate([n_ells, neu_coeffs, poisson_coeffs]):
            
        for iplane in range(4):
            
            # default params
            n_ell, neu_coeff, poisson_coeff = 2000, 0.4, 20
            
            for i, v in enumerate(var):
                if j==0:
                    n_ell = v 
                elif j==1:
                    neu_coeff = v
                elif j==2:
                    poisson_coeff = v
                
                for k, rstr in enumerate(['s2p_0.7', 'caiman']):
                    filename = root / 'sims' / f'results_{rstr}_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}.npy'
                    
                    if filename.exists():
                        #print(f'Loading {filename}')
                        tp, fp, fn, f1 = np.load(filename)
                        tps[j][im, iplane, k, i] = tp
                        fps[j][im, iplane, k, i] = fp
                        fns[j][im, iplane, k, i] = fn

idef = np.nonzero(n_ells == 2000)[0][0]

### simulation example + overlap for figure

In [ ]:
root = Path('/media/carsen/disk1/suite2p_paper/VG23/2025_08_06/1/')
iplane = 1
db = np.load(root / 'suite2p' / f'plane{iplane}' / 'db.npy', allow_pickle=True).item()
reg_outputs = np.load(root / 'suite2p' / f'plane{iplane}' / 'reg_outputs.npy', allow_pickle=True).item()
yrange, xrange = reg_outputs['yrange'], reg_outputs['xrange']
Ly, Lx = db['Ly'], db['Lx']

cp_masks = np.load(root / 'benchmarks' / f'masks_gt_plane{iplane}.npy')
cp_masks = cp_masks[yrange[0]:yrange[1], xrange[0]:xrange[1]]
F_gt = np.load(root / 'benchmarks' / f'F_gt_plane{iplane}.npy')
Fneu_gt = np.load(root / 'benchmarks' / f'Fneu_gt_plane{iplane}.npy')
stat_gt = np.load(root / 'benchmarks' / f'stat_gt_plane{iplane}.npy', allow_pickle=True)
dF_gt = F_gt - 0.7 * Fneu_gt
snr_gt = 1 - 0.5 * np.diff(dF_gt, axis=1).var(axis=1) / dF_gt.var(axis=1)
npix_gt = np.array([len(s['ypix']) for s in stat_gt])
# filter ground-truth and predicted ROIs
min_size = np.percentile(npix_gt, 5)
max_size = np.percentile(npix_gt, 95)
# snr_threshold = 0.25
igood_gt = (snr_gt > 0.25) * (npix_gt > min_size) * (npix_gt < max_size)
dF_gt = dF_gt[igood_gt]

f_ex = []
for ipl in range(4):
    detect_outputs = np.load(root / 'suite2p' / f'plane{ipl}' / 'detect_outputs.npy', allow_pickle=True).item()
    f_ex.append(detect_outputs['max_proj'])
    if ipl==iplane:
        max_proj = f_ex[-1].copy()

minY = np.array([f_ex[i].shape[0] for i in range(len(f_ex))]).min()
minX = np.array([f_ex[i].shape[1] for i in range(len(f_ex))]).min()
f_ex = [f_ex[i][:minY, :minX] for i in range(len(f_ex))]


from cellpose.utils import outlines_list
cp_outlines = outlines_list(cp_masks)

import detect_benchmarks    
import torch 
# defaults
n_ell = 2000
neu_coeff = 0.4
poisson_coeff = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
stat_ell, F_ell = detect_benchmarks.make_dendrites(root, n_planes=4, iplane=iplane, ds=2, n_ell=n_ell)
masks_ell = np.zeros((Ly//2, Lx//2), 'int')
for i in range(len(stat_ell)):
    ypix, xpix = stat_ell[i]['ypix'], stat_ell[i]['xpix']
    masks_ell[ypix, xpix] += 1
S, Fneu_sim = detect_benchmarks.make_neuropil(root, n_planes=4, iplane=ipl, device=device)
Sr = S.reshape(S.shape[0], -1)
Sr = torch.from_numpy(Sr).to(device).float()
with BinaryFile(Ly=Ly//2, Lx=Lx//2, 
                    filename=root / 'suite2p_ds' / f'plane{iplane}' / 'data_filt.bin',
                    ) as f_reg:
    tstart, tend = 3000, 3100
    data = np.array(f_reg[tstart : tend])

    fr_ell = np.zeros(data.shape, 'float32')
    for i in range(len(stat_ell)):
        ypix, xpix, lam = stat_ell[i]['ypix'], stat_ell[i]['xpix'], stat_ell[i]['lam']
        F_ell0 = F_ell[i, tstart : tend, np.newaxis] * (lam / (lam).sum()) * len(ypix)
        fr_ell[:, ypix, xpix] += F_ell0

    fr_ell = torch.from_numpy(fr_ell).to(device) 
    fr_ell = torch.clamp(fr_ell, 0)

    dmean = data.mean()  # compute on first batch
    
    Fsim = torch.from_numpy(Fneu_sim).to(device).float()
    dadd = Fsim[:, tstart : tend].T @ Sr 
    dadd = dadd.reshape(-1, Ly//2, Lx//2)
    dadd += dmean
    dadd = torch.clamp(dadd, 0)    
    
    d_out = torch.from_numpy(data).to(device).float()
    
    d_out *= 1 - neu_coeff
    d_out += neu_coeff * dadd

    d_out += fr_ell * (1 - neu_coeff)

    d_out = torch.poisson(d_out / poisson_coeff) * poisson_coeff
    
    d_out = torch.clamp(d_out, 0, 65534//2).int().cpu().numpy()

neu_ex = dadd.cpu().numpy()
ell_ex = fr_ell.cpu().numpy()

from cellpose.utils import outlines_list, masks_to_outlines

# default params
n_ell, neu_coeff, poisson_coeff = 2000, 0.4, 20

iplane = 1
root = Path(f'/home/carsen/dm11_string/suite2p_paper/hybrid_gt/VG23/')
db = np.load(root / "suite2p_ds" / f"plane{iplane}" / "db.npy", allow_pickle=True).item()
n_frames, Ly, Lx = db["nframes"], db["Ly"], db["Lx"]
reg_outputs = np.load(root / "suite2p_ds" / f"plane{iplane}" / "reg_outputs.npy", allow_pickle=True).item()
yrange, xrange = reg_outputs["yrange"], reg_outputs["xrange"]
    
# load ground-truth ROIs
stat_gt = np.load(root / "benchmarks" / f"stat_gt_plane{iplane}.npy", allow_pickle=True)
F_gt = np.load(root / "benchmarks" / f"F_gt_plane{iplane}.npy", allow_pickle=True)
Fneu_gt = np.load(root / "benchmarks" / f"Fneu_gt_plane{iplane}.npy", allow_pickle=True)
dF_gt = F_gt.copy() - 0.7 * Fneu_gt    

dFs = []
stats = []

rstr = 's2p_0.7'
stats.append(np.load(root / 'sims' / f'stat_{rstr}_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}.npy', allow_pickle=True))
F = np.load(root / 'sims' / f'F_{rstr}_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}.npy')
Fneu = np.load(root / 'sims' / f'Fneu_0.7_s2p_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}.npy')
dFs.append(F - 0.7 * Fneu)

for param_type0 in ['caiman']:
    run_name0 = f'{param_type0}_neu_{neu_coeff:.2f}_ell_{n_ell}_poisson_{poisson_coeff}_plane{iplane}'
    dFs.append(np.load(root / 'sims' / f"F_{run_name0}.npy"))
    stat = np.load(root / 'sims' / f"stat_{run_name0}.npy", allow_pickle=True)
    lam_threshold = 0.1
    for i in range(len(stat)):
        lam = stat[i]["lam"]
        stat[i]["ypix"] = stat[i]["ypix"][lam > lam_threshold * lam.max()]
        stat[i]["xpix"] = stat[i]["xpix"][lam > lam_threshold * lam.max()]
        stat[i]['ypix'] += yrange[0]
        stat[i]['xpix'] += xrange[0]
        stat[i]["lam"] = stat[i]["lam"][lam > lam_threshold * lam.max()]
    stats.append(stat)

snr_threshold = 0.25
snr_gt = 1 - 0.5 * np.diff(dF_gt, axis=1).var(axis=1) / dF_gt.var(axis=1)
npix_gt = np.array([len(s['ypix']) for s in stat_gt])
# filter ground-truth and predicted ROIs
min_size = np.percentile(npix_gt, 5)
max_size = np.percentile(npix_gt, 95)
igood_gt = (snr_gt > snr_threshold) * (npix_gt > min_size) * (npix_gt < max_size)    
stat_gt = stat_gt[igood_gt]
dF_gt = dF_gt[igood_gt]

igoods = []
for dF, stat in zip(dFs, stats):
    snr = 1 - 0.5 * np.diff(dF, axis=1).var(axis=1) / dF.var(axis=1)
    npix = np.array([len(s['ypix']) for s in stat])
    igood = (snr > snr_threshold) * (npix > min_size) * (npix < max_size)
    print(igood.sum())
    igoods.append(igood)

stats = [stat[igood] for stat, igood in zip(stats, igoods)]
    
masks_gt = np.zeros((Ly, Lx), 'uint16')
for i in range(len(stat_gt)):
    ypix, xpix = stat_gt[i]['ypix'], stat_gt[i]['xpix']
    masks_gt[ypix, xpix] = i+1
outlines_gt = masks_to_outlines(masks_gt)

masks_all = []
outlines_all = []
for stat in stats:
    masks = np.zeros((Ly, Lx), 'uint16')
    for i in range(len(stat)):
        ypix, xpix = stat[i]['ypix'], stat[i]['xpix']
        masks[ypix, xpix] = i+1
    outlines = outlines_list(masks)
    masks_all.append(masks)
    outlines_all.append(outlines)

        

In [ ]:
np.save('results/detect_metrics.npy', {
    'max_proj': max_proj,  'cp_outlines': cp_outlines, 'dF_gt': dF_gt, 
    'neu_ex': neu_ex, 'ell_ex': ell_ex,
'f_ex': f_ex, 'd_out': d_out, 'masks_gt': masks_gt,
'outlines_gt': outlines_gt, 'outlines_all': outlines_all, 'tps': tps,
'fps': fps, 'fns': fns, 'idef': idef
})

In [ ]:
import matplotlib
import figures 
import importlib
from fig_utils import *
from cellpose import transforms
importlib.reload(figures)

fig = figures.detectmetrics_fig(max_proj, cp_outlines, dF_gt, neu_ex, 
                          ell_ex, f_ex, d_out,
                          masks_gt, outlines_gt, outlines_all,
                          tps, fps, fns, idef)
fig.savefig('figures/fig5.pdf', dpi=150)

In [ ]:
importlib.reload(figures)
fig = figures.suppfig_detect(fns, fps)
fig.savefig('figures/suppfig_detect.pdf', dpi=150)